**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(s_{0},a_{1},s_{1},...,a_{T},s_{T})=p(s_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=False):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

We use an $\epsilon$-greedy behavioral policy. $\epsilon$ allows us to choose how much we want to explore. Without it we would never learn anything and always exploit under-optimal actions.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2,:]= -1
        self.position[:,:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[self.position < 0] = 0
        self.board[self.x,self.y] = 0
        
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
epochs_train = 300 # set small when debugging
epochs_test = 50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

`position` tells the mouse where it can be (0) or can't be (-1)
`board` tells the mouse where there's cheese (+0.5), poison (-1), or nothing (0)

I modified the environment to fix some issues:
- The rightmost border was missing
- The was cheese and poison under the borders, I set `board` to 0 where `position` is -1
- The mouse was bouncing off the borders when it chose invalid moves, I decided it should rather not move (otherwise, it might "think" it was good or bad to choose an invalid action, if the destination cell is not empty).

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1, epochs + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score += win - lose

        print("Win/lose count {}/{}. Average score ({})".format(win, lose, score / e))
    print('Final score: {}'.format(score / epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=0.3)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix='random')
HTML(display_videos('random1.mp4'))

Win/lose count 10.5/16.0. Average score (-5.5)
Win/lose count 11.5/12.0. Average score (-3.0)
Win/lose count 5.0/10.0. Average score (-3.6666666666666665)
Win/lose count 11.0/17.0. Average score (-4.25)
Win/lose count 7.0/9.0. Average score (-3.8)
Win/lose count 7.0/21.0. Average score (-5.5)
Win/lose count 10.5/14.0. Average score (-5.214285714285714)
Win/lose count 8.5/13.0. Average score (-5.125)
Win/lose count 8.0/14.0. Average score (-5.222222222222222)
Win/lose count 11.0/11.0. Average score (-4.7)
Win/lose count 8.5/21.0. Average score (-5.409090909090909)
Win/lose count 10.0/21.0. Average score (-5.875)
Win/lose count 9.0/10.0. Average score (-5.5)
Win/lose count 11.5/12.0. Average score (-5.142857142857143)
Win/lose count 10.5/14.0. Average score (-5.033333333333333)
Win/lose count 11.5/10.0. Average score (-4.625)
Win/lose count 5.0/12.0. Average score (-4.764705882352941)
Win/lose count 13.5/15.0. Average score (-4.583333333333333)
Win/lose count 4.0/11.0. Average score (-4.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




First, 
\begin{align*}
    Q^\pi(s,a)
    &= \mathbb{E}\left[\sum_t \gamma^{t}r(s_t,a_t)|s_0=s,a_0=a, \forall t \ge 1, a_t=\pi(s_t)\right] \\
    &= r(a, s) + \gamma \mathbb{E}\left[\sum_{t \ge 1} \gamma^{t - 1} r(s_t,a_t)|s_0=s,a_0=a, \forall t \ge 1, a_t=\pi(s_t)\right] \\
    &= r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s' | s_0 = s) \mathbb{E}\left[\sum_{t \ge 1} \gamma^{t - 1} r(s_t,a_t)|s_1=s',a_1=a'=\pi(s), \forall t \ge 1, a_t=\pi(s_t)\right] \\
    &= r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s' | s_0 = s) Q^\pi(s', a' = \pi(s)) \\
    &= E_{(s',a')\sim p(.|s,a)}\left[r(s,a)+\gamma Q^{\pi}(s',a')\right]
\end{align*}

Then,
\begin{align*}
    Q^{*}(s, a)
    &= \max_{\pi} Q^{\pi}(s, a) \\
    &= \max_{\pi} \left[ r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s' | s_0 = s) Q^\pi(s', \pi(s)) \right] \\
    &= r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s' | s_0 = s) \max_{\pi} Q^\pi(s', \pi(s)) \\
    &= r(a, s) + \gamma \sum_{s'} \mathbb{P}(s_1 = s' | s_0 = s) \max_{a'} Q^{*}(s', a') \\
    &= E_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')\right]
\end{align*}

Finally, we have:
\begin{align*}
    & Q^{*}(s, a) = E_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')\right] \\
    &\iff 0 = E_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')\right] -  Q^{*}(s, a) \\
    &\iff 0 = E_{s'\sim \pi^*(.|s,a)}\left[r(s,a)+\gamma\max_{a'}Q^{*}(s',a') -  Q^{*}(s, a) \right]
\end{align*}

Here, the Q-function is some approximation, so we want to minimize the $L^2$-error:
$$
    \mathcal{L}(\theta)
    = E_{s' \sim \pi^*(.|s,a)}\left[ \Vert r(s, a) +\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2} \right]
$$


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
from collections import deque

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen=max_memory)

    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        return self.memory[np.random.randint(0, len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
from keras.layers import Flatten, Input


class DQN(Agent):
    def __init__(self, grid_size,  epsilon=0.1, memory_size=100, batch_size=16, n_state=2):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q = self.model.predict(s.reshape(-1, 5, 5, self.n_state)).ravel()
        return np.argmax(q)

    def reinforce(self, state, next_state, action, reward, game_over):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([state, next_state, action, reward, game_over])
        
        samples = [
            self.memory.random_access()
            for i in range(self.batch_size)
        ]
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        next_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        
        for i in range(self.batch_size):
            state, next_state, action, reward, game_over = samples[i]
            input_states[i] = state
            next_states[i] = next_state
            
        q = self.model.predict(input_states)
        q_hat = self.model.predict(next_states)
        target_q = q.copy()
            
        for i in range(self.batch_size):
            state, next_state, action, reward, game_over = samples[i]            
            target_q[i, action] = reward + self.discount * np.max(q_hat[i])
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential([
            Flatten(input_shape=(5, 5, self.n_state)),
            Dense(16, activation='relu'),
            Dense(4),
        ])
        
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train100.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                816       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 68        
Total params: 884
Trainable params: 884
Non-trainable params: 0
_________________________________________________________________
Epoch 001/300 | Loss 0.0019 | Win/lose count 2.5/4.0 (-1.5)
Epoch 002/300 | Loss 0.0132 | Win/lose count 3.0/2.0 (1.0)
Epoch 003/300 | Loss 0.0075 | Win/lose count 3.5/5.0 (-1.5)
Epoch 004/300 | Loss 0.0112 | Win/lose count 3.5/3.0 (0.5)
Epoch 005/300 | Loss 0.0072 | Win/lose count 5.5/3.0 (2.5)
Epoch 006/300 | Loss 0.0059 | Win/lose count 2.5/0 (2.5)
Epoch 007/300 | Loss 0.0092 | Win/lose count 3.0/2.0 (

Epoch 127/300 | Loss 0.0020 | Win/lose count 15.5/3.0 (12.5)
Epoch 128/300 | Loss 0.0026 | Win/lose count 16.0/1.0 (15.0)
Epoch 129/300 | Loss 0.0009 | Win/lose count 15.5/3.0 (12.5)
Epoch 130/300 | Loss 0.0009 | Win/lose count 12.5/1.0 (11.5)
Epoch 131/300 | Loss 0.0076 | Win/lose count 9.0/1.0 (8.0)
Epoch 132/300 | Loss 0.0032 | Win/lose count 11.0/2.0 (9.0)
Epoch 133/300 | Loss 0.0020 | Win/lose count 19.0/2.0 (17.0)
Epoch 134/300 | Loss 0.0030 | Win/lose count 6.5/0 (6.5)
Epoch 135/300 | Loss 0.0018 | Win/lose count 9.5/2.0 (7.5)
Epoch 136/300 | Loss 0.0018 | Win/lose count 13.0/0 (13.0)
Epoch 137/300 | Loss 0.0028 | Win/lose count 17.5/4.0 (13.5)
Epoch 138/300 | Loss 0.0025 | Win/lose count 21.5/3.0 (18.5)
Epoch 139/300 | Loss 0.0025 | Win/lose count 6.5/0 (6.5)
Epoch 140/300 | Loss 0.0009 | Win/lose count 6.0/2.0 (4.0)
Epoch 141/300 | Loss 0.0020 | Win/lose count 5.5/3.0 (2.5)
Epoch 142/300 | Loss 0.0012 | Win/lose count 17.0/1.0 (16.0)
Epoch 143/300 | Loss 0.0011 | Win/lose coun

Epoch 264/300 | Loss 0.0016 | Win/lose count 15.0/2.0 (13.0)
Epoch 265/300 | Loss 0.0023 | Win/lose count 17.5/0 (17.5)
Epoch 266/300 | Loss 0.0017 | Win/lose count 7.0/4.0 (3.0)
Epoch 267/300 | Loss 0.0033 | Win/lose count 22.5/2.0 (20.5)
Epoch 268/300 | Loss 0.0009 | Win/lose count 13.5/1.0 (12.5)
Epoch 269/300 | Loss 0.0019 | Win/lose count 17.0/2.0 (15.0)
Epoch 270/300 | Loss 0.0036 | Win/lose count 6.5/1.0 (5.5)
Epoch 271/300 | Loss 0.0037 | Win/lose count 6.0/1.0 (5.0)
Epoch 272/300 | Loss 0.0034 | Win/lose count 13.5/0 (13.5)
Epoch 273/300 | Loss 0.0034 | Win/lose count 13.0/1.0 (12.0)
Epoch 274/300 | Loss 0.0023 | Win/lose count 13.5/2.0 (11.5)
Epoch 275/300 | Loss 0.0011 | Win/lose count 3.5/1.0 (2.5)
Epoch 276/300 | Loss 0.0016 | Win/lose count 23.5/3.0 (20.5)
Epoch 277/300 | Loss 0.0032 | Win/lose count 15.0/3.0 (12.0)
Epoch 278/300 | Loss 0.0035 | Win/lose count 8.5/0 (8.5)
Epoch 279/300 | Loss 0.0009 | Win/lose count 20.0/1.0 (19.0)
Epoch 280/300 | Loss 0.0022 | Win/lose c

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [13]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential([
            Conv2D(16, (3, 3), activation='relu', input_shape=(5, 5, self.n_state)),
            Conv2D(8, (3, 3), activation='relu'),
            Flatten(),
            Dense(4),
        ])
        
        model.summary()
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train100.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 3, 16)          304       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_2 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 36        
Total params: 1,500
Trainable params: 1,500
Non-trainable params: 0
_________________________________________________________________
Epoch 001/300 | Loss 0.0149 | Win/lose count 8.0/2.0 (6.0)
Epoch 002/300 | Loss 0.0017 | Win/lose count 3.0/3.0 (0.0)
Epoch 003/300 | Loss 0.0016 | Win/lose count 3.0/11.0 (-8.0)
Epoch 004/300 | Loss 0.0056 | Win/lose count 4.5/3.0 (1.5)
Epoch 005/300 | Loss 0.0017 | Win/

Epoch 124/300 | Loss 0.0015 | Win/lose count 14.5/4.0 (10.5)
Epoch 125/300 | Loss 0.0028 | Win/lose count 16.0/8.0 (8.0)
Epoch 126/300 | Loss 0.0026 | Win/lose count 12.0/7.0 (5.0)
Epoch 127/300 | Loss 0.0014 | Win/lose count 13.5/6.0 (7.5)
Epoch 128/300 | Loss 0.0011 | Win/lose count 15.0/1.0 (14.0)
Epoch 129/300 | Loss 0.0009 | Win/lose count 20.5/9.0 (11.5)
Epoch 130/300 | Loss 0.0015 | Win/lose count 18.5/8.0 (10.5)
Epoch 131/300 | Loss 0.0017 | Win/lose count 14.0/6.0 (8.0)
Epoch 132/300 | Loss 0.0019 | Win/lose count 13.5/3.0 (10.5)
Epoch 133/300 | Loss 0.0019 | Win/lose count 26.5/8.0 (18.5)
Epoch 134/300 | Loss 0.0025 | Win/lose count 11.5/4.0 (7.5)
Epoch 135/300 | Loss 0.0017 | Win/lose count 10.5/6.0 (4.5)
Epoch 136/300 | Loss 0.0017 | Win/lose count 9.0/4.0 (5.0)
Epoch 137/300 | Loss 0.0025 | Win/lose count 21.0/6.0 (15.0)
Epoch 138/300 | Loss 0.0020 | Win/lose count 22.5/2.0 (20.5)
Epoch 139/300 | Loss 0.0006 | Win/lose count 7.0/3.0 (4.0)
Epoch 140/300 | Loss 0.0022 | Win/

Epoch 260/300 | Loss 0.0009 | Win/lose count 20.0/4.0 (16.0)
Epoch 261/300 | Loss 0.0016 | Win/lose count 8.5/5.0 (3.5)
Epoch 262/300 | Loss 0.0008 | Win/lose count 18.5/2.0 (16.5)
Epoch 263/300 | Loss 0.0006 | Win/lose count 11.5/5.0 (6.5)
Epoch 264/300 | Loss 0.0019 | Win/lose count 13.5/2.0 (11.5)
Epoch 265/300 | Loss 0.0007 | Win/lose count 6.0/4.0 (2.0)
Epoch 266/300 | Loss 0.0018 | Win/lose count 6.5/2.0 (4.5)
Epoch 267/300 | Loss 0.0008 | Win/lose count 8.5/7.0 (1.5)
Epoch 268/300 | Loss 0.0019 | Win/lose count 18.5/4.0 (14.5)
Epoch 269/300 | Loss 0.0003 | Win/lose count 18.0/9.0 (9.0)
Epoch 270/300 | Loss 0.0017 | Win/lose count 8.0/7.0 (1.0)
Epoch 271/300 | Loss 0.0012 | Win/lose count 23.5/9.0 (14.5)
Epoch 272/300 | Loss 0.0011 | Win/lose count 12.5/4.0 (8.5)
Epoch 273/300 | Loss 0.0011 | Win/lose count 15.0/6.0 (9.0)
Epoch 274/300 | Loss 0.0011 | Win/lose count 16.0/5.0 (11.0)
Epoch 275/300 | Loss 0.0005 | Win/lose count 16.5/8.0 (8.5)
Epoch 276/300 | Loss 0.0005 | Win/lose 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [27]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 3, 3, 16)          304       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_8 (Flatten)          (None, 8)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 4)                 36        
Total params: 1,500
Trainable params: 1,500
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_12

In [30]:
HTML(display_videos('cnn_test1.mp4'))

In [31]:
HTML(display_videos('fc_test1.mp4'))

The algorithms do not really explore the map: the mouse is very myopic and once it has collected all the rewards in the area, it usually goes back and forth instead of trying to find food in unexplored areas.

Temperature doesn't really affect this behavior.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [37]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1, epoch + 1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            agent.set_epsilon(agent.epsilon * 0.5)

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.malus_position = np.zeros((grid_size,grid_size))
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[:2,:]= -1
        self.position[:, :2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        if train:
            reward += self.malus_position[self.x, self.y]
        
        self.board[self.x, self.y] = 0
        self.malus_position[self.x, self.y] -= 0.01
        
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2,:]= -1
        self.position[:, :2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        
        self.board[self.x,self.y] = 0
        self.board[self.position < 0] = 0
        
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [38]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.3, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore100.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 3, 3, 16)          448       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 1, 1, 8)           1160      
_________________________________________________________________
flatten_11 (Flatten)         (None, 8)                 0         
_________________________________________________________________
dense_15 (Dense)             (None, 4)                 36        
Total params: 1,644
Trainable params: 1,644
Non-trainable params: 0
_________________________________________________________________
Epoch 001/300 | Loss 0.0116 | Win/lose count 8.0/18.090000000000007 (-10.090000000000007)
Epoch 002/300 | Loss 0.0113 | Win/lose count 6.0/18.389999999999993 (-12.389999999999993)
Epoch 003/300 | Loss 0.0237 | Win/lose count 9.0/15.519999999999987 (-6.519999999999987)
Epo

Epoch 088/300 | Loss 0.0022 | Win/lose count 20.0/1.8100000000000012 (18.189999999999998)
Epoch 089/300 | Loss 0.0029 | Win/lose count 22.0/2.5899999999999963 (19.410000000000004)
Epoch 090/300 | Loss 0.0040 | Win/lose count 24.0/1.5100000000000011 (22.49)
Epoch 091/300 | Loss 0.0053 | Win/lose count 22.0/1.7500000000000013 (20.25)
Epoch 092/300 | Loss 0.0045 | Win/lose count 19.5/2.57 (16.93)
Epoch 093/300 | Loss 0.0018 | Win/lose count 27.5/1.1800000000000008 (26.32)
Epoch 094/300 | Loss 0.0020 | Win/lose count 19.5/1.7900000000000014 (17.709999999999997)
Epoch 095/300 | Loss 0.0025 | Win/lose count 27.0/1.8200000000000014 (25.18)
Epoch 096/300 | Loss 0.0028 | Win/lose count 24.0/1.7300000000000013 (22.27)
Epoch 097/300 | Loss 0.0017 | Win/lose count 24.5/1.4200000000000008 (23.08)
Epoch 098/300 | Loss 0.0029 | Win/lose count 20.5/1.8800000000000014 (18.619999999999997)
Epoch 099/300 | Loss 0.0024 | Win/lose count 24.5/2.4200000000000004 (22.08)
Epoch 100/300 | Loss 0.0040 | Win/lose

Epoch 192/300 | Loss 0.0020 | Win/lose count 28.5/1.330000000000001 (27.169999999999998)
Epoch 193/300 | Loss 0.0017 | Win/lose count 21.0/3.319999999999999 (17.68)
Epoch 194/300 | Loss 0.0019 | Win/lose count 21.0/1.350000000000001 (19.65)
Epoch 195/300 | Loss 0.0018 | Win/lose count 24.0/2.0400000000000014 (21.959999999999997)
Epoch 196/300 | Loss 0.0020 | Win/lose count 21.0/1.1500000000000008 (19.849999999999998)
Epoch 197/300 | Loss 0.0024 | Win/lose count 22.5/1.7600000000000011 (20.74)
Epoch 198/300 | Loss 0.0016 | Win/lose count 27.0/1.0200000000000007 (25.98)
Epoch 199/300 | Loss 0.0018 | Win/lose count 27.5/1.0700000000000007 (26.43)
Epoch 200/300 | Loss 0.0026 | Win/lose count 28.0/1.440000000000001 (26.56)
Epoch 201/300 | Loss 0.0031 | Win/lose count 20.0/3.2099999999999995 (16.79)
Epoch 202/300 | Loss 0.0023 | Win/lose count 21.5/5.059999999999996 (16.440000000000005)
Epoch 203/300 | Loss 0.0014 | Win/lose count 23.0/1.7200000000000013 (21.279999999999998)
Epoch 204/300 | 

Epoch 294/300 | Loss 0.0025 | Win/lose count 26.0/1.1800000000000008 (24.82)
Epoch 295/300 | Loss 0.0020 | Win/lose count 22.0/1.500000000000001 (20.5)
Epoch 296/300 | Loss 0.0025 | Win/lose count 27.5/0.9400000000000006 (26.56)
Epoch 297/300 | Loss 0.0029 | Win/lose count 26.5/1.0300000000000007 (25.47)
Epoch 298/300 | Loss 0.0021 | Win/lose count 23.0/1.2400000000000009 (21.759999999999998)
Epoch 299/300 | Loss 0.0022 | Win/lose count 30.0/1.320000000000001 (28.68)
Epoch 300/300 | Loss 0.0027 | Win/lose count 24.0/1.1400000000000008 (22.86)


In [39]:
# Evaluation
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
test(agent,env,10,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore1.mp4'))

Win/lose count 29.0/0. Average score (29.0)
Win/lose count 22.0/0. Average score (25.5)
Win/lose count 23.5/1.0. Average score (24.5)
Win/lose count 25.0/0. Average score (24.625)
Win/lose count 23.0/0. Average score (24.3)
Win/lose count 23.5/0. Average score (24.166666666666668)
Win/lose count 18.5/0. Average score (23.357142857142858)
Win/lose count 23.0/0. Average score (23.3125)
Win/lose count 23.5/0. Average score (23.333333333333332)
Win/lose count 21.0/0. Average score (23.1)
Final score: 23.1


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***